In [1]:
import pandas as pd
import glob
import numpy as np

In [2]:
cornbelters_files = glob.glob("../KCLData/*.csv")
other_files = glob.glob("../CornBeltersData/*.csv")

all_files = cornbelters_files + other_files

# Read and concatenate all CSVs into one DataFrame
stuff_plus = [pd.read_csv(f) for f in all_files]
df = pd.concat(stuff_plus, ignore_index=True)

In [17]:
df_left = df[df['BatterSide'] == 'Left']
df_right = df[df['BatterSide'] == 'Right']

In [18]:
#Combine PitchCall KorBB and PlayResult into one column
def combined_columns(event):
    #If there is a value in KorBB return that since PlayResult sometimes does not have KorBB
    if not pd.isna(event['KorBB']):
        return event['KorBB']
    #If the ball is in play in pitchcall we want to return the result to map it
    if event['PitchCall'] == 'InPlay':
        return event['PlayResult']
    #Since we already have if there's a strikeout or if there's an inplay
    else:
        return event['PitchCall']

In [19]:
def validate_data(df):
    """Validate DataFrame columns and data types."""
    required_columns = ['GameID', 'Inning', 'PAofInning', 'Outs', 'OutsOnPlay', 'RunsScored', 'pitch_by_pitch', 'Balls', 'Strikes']
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Check for missing values
    for col in required_columns:
        if df[col].isna().any():
            print(f"Warning: {col} contains {df[col].isna().sum()} missing values")
    
    # Validate data types and ranges
    if not df['PAofInning'].apply(lambda x: pd.isna(x) or (isinstance(x, (int, float)) and x >= 1)).all():
        raise ValueError("PAofInning must be numeric and >= 1 or NaN")
    if not df['Outs'].isin([0, 1, 2, np.nan]).all():
        print(f"Warning: Invalid values in Outs. Expected 0, 1, 2, or NaN. Found: {df['Outs'].unique()}")
    if not df['OutsOnPlay'].apply(lambda x: pd.isna(x) or (isinstance(x, (int, float)) and x >= 0)).all():
        raise ValueError("OutsOnPlay must be numeric and >= 0 or NaN")
    if not df['RunsScored'].apply(lambda x: pd.isna(x) or (isinstance(x, (int, float)) and x >= 0)).all():
        raise ValueError("RunsScored must be numeric and >= 0 or NaN")
    if not df['Balls'].isin([0, 1, 2, 3, 4, np.nan]).all():
        print(f"Warning: Invalid values in Balls. Expected 0-4 or NaN. Found: {df['Balls'].unique()}")
    if not df['Strikes'].isin([0, 1, 2, np.nan]).all():
        print(f"Warning: Invalid values in Strikes. Expected 0-2 or NaN. Found: {df['Strikes'].unique()}")

In [20]:
def infer_base_state(df_inning):
    """Infer base state (None or Runners) based on prior plays in the inning."""
    if df_inning.empty:
        return pd.Series(['None'] * len(df_inning), index=df_inning.index)
    
    base_state = []
    runners_on = False
    df_inning = df_inning.sort_values('PAofInning')  # Ensure sorted order
    
    for i, (idx, row) in enumerate(df_inning.iterrows()):
        # Reset runners at the start of an inning or after 3 outs
        if row['PAofInning'] == 1 or (i > 0 and df_inning.iloc[i-1]['Outs'] == 2):
            runners_on = False
        # Outcomes that likely add runners (non-outs)
        if row['pitch_by_pitch'] in ['Single', 'Double', 'Triple', 'Walk', 'HitByPitch', 'Error', 'FieldersChoice', 'IntentionalWalk']:
            runners_on = True
        # Outcomes that clear bases or end inning
        elif row['pitch_by_pitch'] in ['HomeRun', 'Strikeout', 'Out', 'Sacrifice']:
            runners_on = False
        base_state.append('Runners' if runners_on else 'None')
    
    return pd.Series(base_state, index=df_inning.index)

In [21]:
def create_run_expectancy_matrix(df):
    """
    Create a run expectancy matrix for college baseball data.
    
    Parameters:
    df (pd.DataFrame): DataFrame with 'GameID', 'Inning', 'PAofInning', 'Outs', 'OutsOnPlay', 'RunsScored', 'pitch_by_pitch'.
    
    Returns:
    pd.DataFrame: Run expectancy matrix with mean runs per base-out state.
    pd.DataFrame: Updated DataFrame with base_state and base_out_state columns.
    """
    # Validate input data
    validate_data(df)
    
    # Create a copy and clean data
    df = df.copy()
    df['GameID'] = df['GameID'].astype(str)
    df['Inning'] = df['Inning'].astype(str)
    df = df.dropna(subset=['GameID', 'Inning', 'PAofInning'])
    df['Outs'] = df['Outs'].fillna(0).astype(int)
    df['OutsOnPlay'] = df['OutsOnPlay'].fillna(0).astype(int)
    df['RunsScored'] = df['RunsScored'].fillna(0).astype(float)
    df['Balls'] = df['Balls'].fillna(0).astype(int)
    df['Strikes'] = df['Strikes'].fillna(0).astype(int)
    non_scoring = ['Foul', 'BallCalled', 'StrikeCalled', 'StrikeSwinging', 'FoulTip']
    df.loc[df['pitch_by_pitch'].isin(non_scoring), 'RunsScored'] = 0
    df.loc[df['pitch_by_pitch'] == 'StrikeoutSwinging', 'pitch_by_pitch'] = 'Strikeout'
    
    # Create inning identifier
    df['inning_id'] = df['GameID'] + '_' + df['Inning']
    
    # Reindex PAofInning to ensure sequential values
    df['PAofInning'] = df.groupby('inning_id').cumcount() + 1
    
    # Sort by inning and PAofInning
    df = df.sort_values(['inning_id', 'PAofInning'])
    
    # Infer base state for each pitch
    try:
        df['base_state'] = df.groupby('inning_id', group_keys=False).apply(infer_base_state)
    except Exception as e:
        print(f"Error in infer_base_state: {e}")
        for inning_id, group in df.groupby('inning_id'):
            if not group['PAofInning'].is_monotonic_increasing or group['PAofInning'].min() < 1:
                print(f"Problematic inning_id: {inning_id}")
                print(group[['PAofInning', 'Outs', 'OutsOnPlay', 'pitch_by_pitch', 'RunsScored']])
        raise
    
    # Create base-out state
    df['base_out_state'] = df['base_state'] + '_' + df['Outs'].astype(str)
    
    # Calculate cumulative runs to end of inning
    runs_per_play = df.groupby(['inning_id', 'PAofInning', 'base_out_state'])['RunsScored'].sum().reset_index()
    runs_per_play['runs_to_end'] = runs_per_play.groupby('inning_id')['RunsScored'].transform(lambda x: x[::-1].cumsum()[::-1])
    
    # Compute mean run expectancy for each base-out state
    re_matrix = runs_per_play.groupby('base_out_state')['runs_to_end'].mean().reset_index()
    re_matrix.rename(columns={'runs_to_end': 'run_expectancy'}, inplace=True)
    
    return re_matrix, df

In [22]:
def calculate_delta_run_exp(df, re_matrix):
    """
    Calculate delta run expectancy for each pitch event using OutsOnPlay.
    
    Parameters:
    df (pd.DataFrame): DataFrame with pitch data and base_out_state.
    re_matrix (pd.DataFrame): Run expectancy matrix.
    
    Returns:
    pd.DataFrame: DataFrame with added 'delta_run_exp' column.
    """
    df = df.copy()
    df['start_base_out_state'] = df['base_out_state']
    
    # Infer end state after pitch
    df['end_outs'] = df['Outs'] + df['OutsOnPlay']
    df['end_outs'] = df['end_outs'].clip(upper=3)
    df['end_base_state'] = df['base_state']
    
    # Update base state for outcomes that clear or add runners
    df.loc[df['pitch_by_pitch'].isin(['HomeRun', 'Strikeout', 'Out', 'Sacrifice']), 'end_base_state'] = 'None'
    df.loc[df['pitch_by_pitch'].isin(['Single', 'Double', 'Triple', 'Walk', 'HitByPitch', 'Error', 'IntentionalWalk']), 'end_base_state'] = 'Runners'
    
    # Create end base-out state
    df['end_base_out_state'] = df['end_base_state'] + '_' + df['end_outs'].astype(str)
    
    # Merge start and end run expectancy
    df = df.merge(re_matrix, left_on='start_base_out_state', right_on='base_out_state', how='left')
    df.rename(columns={'run_expectancy': 'start_re'}, inplace=True)
    df = df.merge(re_matrix, left_on='end_base_out_state', right_on='base_out_state', how='left')
    df.rename(columns={'run_expectancy': 'end_re'}, inplace=True)
    
    # Handle missing run expectancy values
    df['start_re'] = df['start_re'].fillna(0)
    df['end_re'] = df['end_re'].fillna(0)
    
    # Calculate delta run expectancy
    df['delta_run_exp'] = df['end_re'] - df['start_re'] + df['RunsScored']
    
    # Drop temporary columns
    df.drop(['start_re', 'end_re', 'base_out_state_x', 'base_out_state_y', 'end_outs', 'end_base_state', 'end_base_out_state'], axis=1, inplace=True)
    
    return df

In [23]:
def calculate_mean_delta_run_exp(df):
    """
    Calculate mean delta run expectancy by pitch outcome and count.
    
    Parameters:
    df (pd.DataFrame): DataFrame with 'pitch_by_pitch', 'delta_run_exp', 'Balls', 'Strikes'.
    
    Returns:
    pd.DataFrame: DataFrame with mean delta_run_exp for each outcome and count.
    """
    mean_delta_run_exp = df.groupby(['TaggedPitchType', 'Balls', 'Strikes'])['delta_run_exp'].mean().reset_index()
    mean_delta_run_exp.rename(columns={'delta_run_exp': 'delta_run_exp_mean'}, inplace=True)
    return mean_delta_run_exp

In [24]:
def build_delta_run_exp_df(df):
    """
    Build DataFrame with delta run expectancy for each pitch.
    
    Parameters:
    df (pd.DataFrame): DataFrame with 'GameID', 'Inning', 'PAofInning', 'Outs', 'OutsOnPlay', 'RunsScored', 
                       'pitch_by_pitch', 'Balls', 'Strikes'.
    
    Returns:
    pd.DataFrame: DataFrame with 'delta_run_exp' and 'delta_run_exp_mean' columns.
    pd.DataFrame: Run expectancy matrix.
    pd.DataFrame: Mean delta run expectancy by outcome and count.
    """
    # Create run expectancy matrix and update DataFrame with base states
    re_matrix, df_with_base_states = create_run_expectancy_matrix(df)
    
    # Calculate delta run expectancy for each pitch
    df_with_delta = calculate_delta_run_exp(df_with_base_states, re_matrix)
    
    # Calculate mean delta run expectancy by outcome and count
    mean_delta_run_exp_df = calculate_mean_delta_run_exp(df_with_delta)
    
    # Merge mean delta run expectancy back into DataFrame
    df_final = df_with_delta.merge(mean_delta_run_exp_df, on=['TaggedPitchType', 'Balls', 'Strikes'], how='left')
    
    # Handle missing mean delta run expectancy values
    df_final['delta_run_exp_mean'] = df_final['delta_run_exp_mean'].fillna(0)
    
    return df_final, re_matrix, mean_delta_run_exp_df

In [ ]:
df_left['pitch_by_pitch'] = df_left.apply(combined_columns,axis=1)
df_right['pitch_by_pitch'] = df_right.apply(combined_columns,axis=1)

In [ ]:
df_left = df_left.dropna(subset=['Outs','pitch_by_pitch','PAofInning','GameID','TaggedPitchType'])
df_right = df_right.dropna(subset=['Outs','pitch_by_pitch','PAofInning','GameID','TaggedPitchType'])


In [26]:
df_left['OutsOnPlay'] = df_left['OutsOnPlay'].fillna(0)
df_left['RunsScored'] = df_left['RunsScored'].fillna(0)
df_right['OutsOnPlay'] = df_right['OutsOnPlay'].fillna(0)
df_right['RunsScored'] = df_right['RunsScored'].fillna(0)

C:\Users\isu_mvquirk\AppData\Local\Temp\ipykernel_26172\242118967.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left['OutsOnPlay'] = df_left['OutsOnPlay'].fillna(0)
C:\Users\isu_mvquirk\AppData\Local\Temp\ipykernel_26172\242118967.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left['RunsScored'] = df_left['RunsScored'].fillna(0)
C:\Users\isu_mvquirk\AppData\Local\Temp\ipykernel_26172\242118967.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [27]:
df_final, re_matrix, mean_delta_run_exp_df = build_delta_run_exp_df(df_left)
df_final.to_csv('left_college_pitch_pitch_data_with_delta_run_exp.csv', index=False)
re_matrix.to_csv('left_college_run_pitch_expectancy_matrix.csv', index=False)
mean_delta_run_exp_df.to_csv('left_mean_pitch_delta_run_exp_by_outcome_count.csv', index=False)

C:\Users\isu_mvquirk\AppData\Local\Temp\ipykernel_26172\3406481458.py:40: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['base_state'] = df.groupby('inning_id', group_keys=False).apply(infer_base_state)


In [28]:
df_final, re_matrix, mean_delta_run_exp_df = build_delta_run_exp_df(df_right)
df_final.to_csv('right_college_pitch_pitch_data_with_delta_run_exp.csv', index=False)
re_matrix.to_csv('right_college_run_pitch_expectancy_matrix.csv', index=False)
mean_delta_run_exp_df.to_csv('right_mean_pitch_delta_run_exp_by_outcome_count.csv', index=False)

C:\Users\isu_mvquirk\AppData\Local\Temp\ipykernel_26172\3406481458.py:40: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['base_state'] = df.groupby('inning_id', group_keys=False).apply(infer_base_state)


In [ ]:
features = ['PlateLocHeight','PlateLocSide','pitch_by_pitch','strikes','balls','outs']